In [1]:
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import numpy as np
import re
import string
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model

/home/kalyan/ds/ds_env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/kalyan/ds/ds_env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Reading entire data

In [52]:
total_data = pd.read_csv('/home/kalyan/Downloads/consumer-reviews-of-amazon-products/1429_1.csv')
total_data.describe()

/home/kalyan/ds/ds_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,reviews.id,reviews.numHelpful,reviews.rating,reviews.userCity,reviews.userProvince
count,1.0,34131.000000,34627.000000,0.0,0.0
mean,111372787.0,0.630248,4.584573,NaN,NaN
std,NaN,13.215775,0.735653,NaN,NaN
min,111372787.0,0.000000,1.000000,NaN,NaN
25%,111372787.0,0.000000,4.000000,NaN,NaN
50%,111372787.0,0.000000,5.000000,NaN,NaN
75%,111372787.0,0.000000,5.000000,NaN,NaN
max,111372787.0,814.000000,5.000000,NaN,NaN


#  Getting null value count

In [28]:
data_considered = total_data[['reviews.rating' , 'reviews.text']]
print(data_considered.isnull().sum()) #Checking for null values

reviews.rating    33
reviews.text       1
dtype: int64


# Removing all null vales  in total data

In [29]:
data_with_out_nulls = data_considered[data_considered['reviews.rating'].notnull()]
data_with_out_nulls = data_with_out_nulls[data_with_out_nulls['reviews.text'].notnull()]

#  making sure that nulls were removed

In [30]:
data_with_out_nulls.isnull().sum()

reviews.rating    0
reviews.text      0
dtype: int64

# categorize the data into types positive and negitive 

In [31]:
data_with_out_nulls["status"] = data_with_out_nulls["reviews.rating"]>=4

data_with_out_nulls["status"] = data_with_out_nulls["status"].replace([True , False] , ["pos" , "neg"])

# Converting all text into lower and removing all special chatachers

In [32]:
def cleanup_data(text):
    text = str(text)
    text = text.lower()
    text = text.encode('utf-8').decode('ascii', 'ignore')
    return text

In [33]:
data_with_out_nulls["clean_text"] = data_with_out_nulls["reviews.text"].apply(cleanup_data)

In [34]:
data_with_out_nulls['clean_text']

0        this product so far has not disappointed. my c...
1        great for beginner or experienced person. boug...
2        inexpensive tablet for him to use and learn on...
3        i've had my fire hd 8 two weeks now and i love...
4        i bought this for my grand daughter when she c...
5        this amazon fire 8 inch tablet is the perfect ...
6        great for e-reading on the go, nice and light ...
7        i gave this as a christmas gift to my inlaws, ...
8        great as a device to read books. i like that i...
9        i love ordering books and reading them with th...
10       not easy for elderly users cease of ads that p...
11       excellent product. easy to use, large screen m...
12       wanted my father to have his first tablet and ...
13       simply does everything i need. thank youand si...
14       got it as a present and love the size of the s...
15       the kindle is easiest to use, graphics and scr...
16       nice reader. almost perfect for what i want/ne.

# spliting train and test data

In [35]:
final_data = data_with_out_nulls[["clean_text" , "status"]]
train=final_data.sample(frac=0.9,random_state=2018)
test=final_data.drop(train.index)

In [36]:
train.shape

(31163, 2)

In [37]:
test.shape

(3463, 2)

In [53]:
commonWords = stopwords.words('english')

In [54]:
commonWords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [14]:
commonWords = stopwords.words('english')
commonWords.remove('not')

count_vect = CountVectorizer(min_df=2 ,stop_words=commonWords , ngram_range=(1,2))
tfidf_transformer = TfidfTransformer()



In [15]:
X_train_counts = count_vect.fit_transform(train["clean_text"])        
X_train = tfidf_transformer.fit_transform(X_train_counts)


X_test_counts = count_vect.transform(test["clean_text"])
X_test = tfidf_transformer.transform(X_test_counts)


In [17]:

logreg = linear_model.LogisticRegression(solver='lbfgs' , C=1000)
logistic = logreg.fit(X_train, train["status"])
print(logreg.score(X_test , test["status"]))


0.938492636442391


In [18]:
data_with_nulls = data_considered[data_considered['reviews.rating'].isnull()]

In [22]:
data_with_nulls["clean_text"] = data_with_nulls["reviews.text"].apply(cleanup_data)

/home/kalyan/ds/ds_env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [39]:
data_with_nulls_counts = count_vect.transform(data_with_nulls["clean_text"])
missing_data = tfidf_transformer.transform(data_with_nulls_counts)


In [41]:
data_with_nulls['status'] = logreg.predict(missing_data)